### 請解壓縮data.rar，取得本程式之數據

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import codecs
from modules import embed , normalize , conv1d , conv1d_banks , gru , prenet , highwaynet
from hyperparams import Hyperparams as hp
from sklearn.model_selection import train_test_split
import re
from xpinyin import Pinyin 
import math
import random

## 數據預處理

In [2]:
# def clean(text):
#     if re.search('[A-Za-z0-9]', text) is None: # 只要有英文或數字的sent，全部略過
#         text = re.sub('[^\u4e00-\u9fa5。，！？]' , '' , text) # 只要是"漢字。，！？"以外的詞換都換成''
#         text = text.replace(' ' , '')
#         return text
#     else:    
#         return ''

# def align(sent):
    
#     pinyin = Pinyin()
#     pnyns = pinyin.get_pinyin(sent , ' ').split()
    
#     # 所有不足拼音的漢字都要加上'_'直到長度等於該漢字對應的拼音
#     # 漢字 : [大_] [巴_] [的_] [门__] [能__]  [将____] [人__] [夹__] [死_]
#     # 拼音 : [da]  [ba]  [de]  [men]  [nen]  [gjiang] [ren]  [jia]  [si]
#     hanzis = []
#     for i , word in enumerate(sent):
#         while len(word) < len(pnyns[i]):
#             word += '_'
#             if len(word) == len(pnyns[i]):
#                 break
#         hanzis.append(word)
    
#     pnyns = ''.join(pnyns)
#     hanzis = ''.join(hanzis)
    
#     return pnyns , hanzis

# fout = codecs.open('data/zh.tsv' , 'w' , 'utf-8')
# fin = codecs.open('data/zho_news_2007-2009_1M-sentences.txt' , 'r' , 'utf-8')
# lines = fin.readlines()  
# for i , line in enumerate(lines):         
#     idx , sent = line.strip().split('\t')
#     sent = clean(sent)
#     if len(sent) > 0:
#         try:
#             pnyns , hanzis = align(sent)
#             fout.write('{}\t{}\t{}\n'.format(idx , pnyns , hanzis))
#         except:
#             print('這一句有問題 : {}'.format(line))
#             pass

## 讀取數據

In [3]:
fin = open('data/zh.tsv', 'r' , encoding = 'utf-8')
lines = fin.readlines()

source_data , target_data = [] , []
for line in lines:
    try:
        _  , pnyn_sents , hanzi_sents = line.strip().split('\t')
    except ValueError:
        continue

    # 我很無聊，他也很無聊。大家都很無聊。 
    # 我很無聊，|他也很無聊。|大家都很無聊。|
    pnyn_sents = re.sub('(?<=[。，！？])' , r'|' , pnyn_sents).split('|')  # 在。，！？後面加上|，比較好從這些標點符號的地方將整個句子分開
    hanzi_sents = re.sub('(?<=[。，！？])' , r'|' , hanzi_sents).split('|')  # 在。，！？後面加上|，比較好從這些標點符號的地方將整個句子分開

    for pnyn_sent , hanzi_sent in zip(pnyn_sents , hanzi_sents):   
        assert len(pnyn_sent) == len(hanzi_sent)
        if 10 < len(pnyn_sent) <= 50: # 考慮長度在[10 , 50)句子     
            x = [pnyn for pnyn in pnyn_sent]
            y = [hanzi for hanzi in hanzi_sent] 
            if len(source_data) < 80000: # 只收集80000筆資料
                source_data.append(x)
                target_data.append(y)           
    if len(source_data) == 80000: # 收集80000筆資料，停止迴圈
        break

In [4]:
# 建立拼音的詞庫
pnyns = 'EUabcdefghijklmnopqrstuvwxyz0123456789。，！？' # E: Empty , U: Unknown
pnyn_to_int = {}
for idx , pnyn in enumerate(pnyns):
    pnyn_to_int[pnyn] = idx
int_to_pnyn = dict(zip(pnyn_to_int.values() , pnyn_to_int.keys()))   

# 建立漢字的詞庫
counter = {}
for hanzi_sent in target_data:
    for word in hanzi_sent:
        if word in counter.keys():
            counter[word] += 1
        elif word not in counter.keys():   
            counter[word] = 1
hanzis = [hanzi for hanzi , cnt in counter.items() if cnt > 5] # 把詞頻小於5的全部刪除
hanzis.remove('_')
hanzis = ['E' , 'U' , '_' ] + hanzis  # 0 : Empty->E , 1 : Unknown->U , 2 : Blank->B
hanzi_to_int = {hanzi : int_ for int_ , hanzi in enumerate(hanzis)}
int_to_hanzi = dict(zip(hanzi_to_int.values() , hanzi_to_int.keys()))

In [5]:
# 決定seq的最大長度 
max_seq_len = max([len(x) for x in source_data])

# 將source_temp與target_temp中的字母全部換成index     
source_pad , target_pad = [] , []
for x , y in zip(source_data , target_data):
    source_temp , target_temp = [] , []    
    for x_i , y_i in zip(x , y):
        if x_i not in pnyn_to_int.keys(): source_temp.append(1) # 1 -> unknown
        else: source_temp.append(pnyn_to_int[x_i])
        if y_i not in hanzi_to_int.keys(): target_temp.append(1) # 1 -> unknown
        else: target_temp.append(hanzi_to_int[y_i])
    
    # 補0補到長度為max_seq_len
    while len(source_temp) < max_seq_len:
        source_temp.append(0)  # 0 -> empty
        target_temp.append(0)  # 0 -> empty
                
    source_pad.append(source_temp)  
    target_pad.append(target_temp)  
            
source_pad = np.array(source_pad).astype(int) 
target_pad = np.array(target_pad).astype(int)  

# 將數據分為訓練集與測試集
(trainX , testX , trainY , testY) = train_test_split(source_pad ,
                                                     target_pad , 
                                                     test_size = 0.01 , 
                                                     random_state = 42)

## Build Model

In [6]:
source = tf.placeholder(tf.int32 , [None , max_seq_len] , name = 'source')
target = tf.placeholder(tf.int32 , [None , max_seq_len] , name = 'target')
target_onehot = tf.one_hot(tf.reshape(target , [-1]) , depth = len(hanzi_to_int))
on_train = tf.placeholder(tf.bool) # train/test selector for batch normalisation

In [7]:
# Character Embedding for x
enc = embed(source , len(pnyn_to_int) , hp.embed_size , scope = 'emb_x')

# Encoder pre-net
prenet_out = prenet(enc,
                    num_units = [hp.embed_size , hp.embed_size // 2],
                    is_training = True) 

## Conv1D bank => 考慮不同捲積長度，長度越長代表看到的上下文越多，長度越短代表看到的上下文越少，最後可分別將不同捲積長度的輸出堆疊起來
enc , ema_0 = conv1d_banks(prenet_out ,
                           on_train ,
                           K = hp.encoder_num_banks ,
                           num_units = hp.embed_size // 2)  

## Max pooling
enc = tf.layers.max_pooling1d(enc , 2 , 1 , padding = 'same') 

## Conv1D projections
enc = conv1d(enc , hp.embed_size // 2 , 5, scope = 'conv1d_1')

enc , ema_1 = normalize(enc , 
                        on_train , 
                        type = hp.norm_type , 
                        activation_fn = tf.nn.relu , 
                        scope = 'norm1')

enc = conv1d(enc , hp.embed_size // 2 , 5 , scope='conv1d_2')
  
enc , ema_2 = normalize(enc ,
                        on_train , 
                        type = hp.norm_type , 
                        activation_fn = None ,
                        scope = 'norm2')

enc += prenet_out  

## Highway Nets
for i in range(hp.num_highwaynet_blocks):
    enc = highwaynet(enc , num_units = hp.embed_size // 2,
                     scope = 'highwaynet_{}'.format(i))  

## Bidirectional GRU
enc = gru(enc , hp.embed_size // 2 , True , scope = 'gru1')  

## Readout
outputs = tf.layers.dense(enc , len(hanzi_to_int) , use_bias = False)
logits = tf.reshape(outputs , [-1 , len(hanzi_to_int)])

## predicting_logits只是要觀察結果，不列入計算
predicting_logits = tf.nn.softmax(logits)
predicting_logits = tf.argmax(predicting_logits , axis = 1)
predicting_logits = tf.reshape(predicting_logits , [-1 , max_seq_len])

## compute loss
loss = tf.nn.softmax_cross_entropy_with_logits(labels = target_onehot , logits = logits)
total_loss = tf.reduce_mean(loss)

## optimizer
update_ema = tf.group([ema_0 , ema_1 , ema_2])
optimizer = tf.train.AdamOptimizer(learning_rate = hp.lr)
train_op = optimizer.minimize(total_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



## Training

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

num_batch = math.ceil(len(source_pad)/ hp.batch_size)
train_loss_mean_record = []
val_loss_record = []
for epoch_i in range(0 , hp.num_epochs): 
    # shuffle index
    index = np.arange(len(trainX))
    np.random.shuffle(index)
    trainX = trainX[index]
    trainY = trainY[index]
     
    # 在每進行一個epoch前，把每個batch的index先決定出來
    batch_index = []
    temp = []
    count = 0 # 隨機決定index的開頭 
    while len(batch_index) <= num_batch:  # 1個batch裡只有num_batch筆資料
        temp.append(count)
        count += 1
        if len(temp) == hp.batch_size:
            batch_index.append(temp)
            temp = []
        if count == len(trainX):
            count = 0
    
    train_loss_temp = []        
    for batch_i in range(0 , num_batch):
        source_pad_batch , target_pad_batch =\
        trainX[batch_index[batch_i] , :] , trainY[batch_index[batch_i] , :] 
        
        feed_dict = {source : source_pad_batch , 
                     target : target_pad_batch , 
                     on_train : True}
        
        _ , _ , training_loss , predicting_value = sess.run([train_op , update_ema , total_loss , predicting_logits] , feed_dict)
        train_loss_temp.append(training_loss) 

        idx = random.randint(0 , hp.batch_size - 1) # 決定想要查看的index
        source_value = source_pad_batch[idx , :]
        target_value = target_pad_batch[idx , :]
        predicting_value = predicting_value[idx , :]

        source_result = ''
        for int_source in source_value:
            if int_to_pnyn[int_source] == 'E' : break
            source_result += int_to_pnyn[int_source]

        target_result = ''
        for int_target in target_value:
            if int_to_hanzi[int_target] == 'E' : break
            target_result += int_to_hanzi[int_target]

        predicting_result = ''
        for int_predicting in predicting_value:
            if int_to_hanzi[int_predicting] == 'E' : break
            predicting_result += int_to_hanzi[int_predicting]

        if batch_i % 200 == 0:
            print('=' * 70)
            print('Epoch : {}/{}'.format(epoch_i , hp.num_epochs))
            print('Batch : {}/{}'.format(batch_i , num_batch))
            print('Training Loss : {:.3f}'.format(training_loss))
            print('source_value : {}'.format(source_result))
            print('target_value : {}'.format(target_result))
            print('predicting_value : {}\n'.format(predicting_result))
    
    feed_dict = {source : testX , target : testY , on_train : False}
    val_loss = sess.run(total_loss , feed_dict)    
    print('*' * 30) 
    print('Epoch : {}/{}'.format(epoch_i , hp.num_epochs))               
    print('train_loss_batch_mean : {:.2f}'.format(np.array(train_loss_temp).mean()))
    print('val_loss : {:.2f}'.format(val_loss))
    print('*' * 30 , '\n')  
    train_loss_mean_record.append(np.array(train_loss_temp).mean())
    val_loss_record.append(val_loss)

Epoch : 0/30
Batch : 0/400
Training Loss : 8.098
source_value : yidanfashengjiufen，
target_value : 一_旦__发_生____纠__纷__，
predicting_value : 胎坞坞冻果果果果果果果果果果果果穆果果果药药果果果果果果药果果果果药果果果果果果果果果果果果果果果村



In [ ]:
# save the model for checkpoint
saver = tf.train.Saver()
saver.save(sess , './model/' , global_step = epoch_i)

In [ ]:
# 看一下testX的預測結果
feed_dict = {source : testX , on_train : False}
predicting_value_valid = sess.run(predicting_logits , feed_dict)

for source_value , target_value , predicting_value in zip(testX , testY , predicting_value_valid):

    source_result_valid = ''
    for int_source in source_value:
        if int_to_pnyn[int_source] == 'E' : break
        source_result_valid += int_to_pnyn[int_source]

    target_result_valid = ''
    for int_target in target_value:
        if int_to_hanzi[int_target] == 'E' : break
        target_result_valid += int_to_hanzi[int_target]

    predicting_result_valid = ''
    for int_predicting in predicting_value:
        if int_to_hanzi[int_predicting] == 'E' : break
        predicting_result_valid += int_to_hanzi[int_predicting]
             
    print('=' * 30)
    print('source_value : {}'.format(source_result_valid))
    print('target_value : {}'.format(target_result_valid))
    print('predicting_value : {}'.format(predicting_result_valid))